In [3]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
%cd 'gdrive/MyDrive/Colab Notebooks/dataset'

/content/gdrive/MyDrive/Colab Notebooks/dataset


In [5]:
import pandas as pd

df = pd.read_csv('weather_dataset.csv')

In [6]:
df.head()

,dt,dt_iso,timezone,city_name,lat,lon,temp,temp_min,temp_max,pressure,...,snow_3h,snow_6h,snow_12h,snow_24h,snow_today,clouds_all,weather_id,weather_main,weather_description,weather_icon
0,631152000,1990-01-01 00:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,25.82,25.82,25.82,1010.0,...,NaN,NaN,NaN,NaN,NaN,98,500,Rain,light rain,10d
1,631155600,1990-01-01 01:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,26.20,26.20,26.20,1011.0,...,NaN,NaN,NaN,NaN,NaN,91,500,Rain,light rain,10d
2,631159200,1990-01-01 02:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,26.45,26.45,26.45,1011.0,...,NaN,NaN,NaN,NaN,NaN,94,500,Rain,light rain,10d
3,631162800,1990-01-01 03:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,26.80,26.80,26.80,1011.0,...,NaN,NaN,NaN,NaN,NaN,94,500,Rain,light rain,10d
4,631166400,1990-01-01 04:00:00 +0000 UTC,28800,Denpasar,-8.652497,115.219117,27.04,27.04,27.04,1010.0,...,NaN,NaN,NaN,NaN,NaN,76,803,Clouds,broken clouds,04d


In [7]:
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264924 entries, 0 to 264923
Data columns (total 32 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   dt                   264924 non-null  int64  
 1   dt_iso               264924 non-null  object 
 2   timezone             264924 non-null  int64  
 3   city_name            264924 non-null  object 
 4   lat                  264924 non-null  float64
 5   lon                  264924 non-null  float64
 6   temp                 264924 non-null  float64
 7   temp_min             264924 non-null  float64
 8   temp_max             264924 non-null  float64
 9   pressure             264924 non-null  float64
 10  sea_level            0 non-null       float64
 11  grnd_level           0 non-null       float64
 12  humidity             264924 non-null  int64  
 13  wind_speed           264924 non-null  float64
 14  wind_deg             264924 non-null  int64  
 15  rain_1h          

In [8]:
df.shape

(264924, 32)

In [9]:
df.isnull().sum()

dt                          0
dt_iso                      0
timezone                    0
city_name                   0
lat                         0
lon                         0
temp                        0
temp_min                    0
temp_max                    0
pressure                    0
sea_level              264924
grnd_level             264924
humidity                    0
wind_speed                  0
wind_deg                    0
rain_1h                248638
rain_3h                248768
rain_6h                228826
rain_12h               264912
rain_24h               258107
rain_today             264924
snow_1h                264924
snow_3h                264924
snow_6h                264924
snow_12h               264924
snow_24h               264924
snow_today             264924
clouds_all                  0
weather_id                  0
weather_main                0
weather_description         0
weather_icon                0
dtype: int64

In [10]:
df.drop(['timezone', 'city_name', 'lat', 'lon','pressure', 'rain_1h',
       'rain_3h', 'rain_6h', 'rain_12h', 'rain_24h', 'rain_today', 'snow_1h',
       'snow_3h', 'snow_6h', 'snow_12h', 'snow_24h', 'snow_today',
       'clouds_all', 'weather_id','weather_icon'], axis=1, inplace=True)

In [11]:
df['date'] = pd.to_datetime(df['dt_iso'], errors='coerce')
df = df.sort_values(by='date')

<ipython-input-11-053fcecb544a>:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['dt_iso'], errors='coerce')


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264924 entries, 0 to 264923
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   dt                   264924 non-null  int64         
 1   dt_iso               264924 non-null  object        
 2   temp                 264924 non-null  float64       
 3   temp_min             264924 non-null  float64       
 4   temp_max             264924 non-null  float64       
 5   sea_level            0 non-null       float64       
 6   grnd_level           0 non-null       float64       
 7   humidity             264924 non-null  int64         
 8   wind_speed           264924 non-null  float64       
 9   wind_deg             264924 non-null  int64         
 10  weather_main         264924 non-null  object        
 11  weather_description  264924 non-null  object        
 12  date                 0 non-null       datetime64[ns]
dtypes: datetime64[

In [13]:
# Ekstrak data suhu harian
temperatures = df['temp'].tolist()

In [14]:
# Fungsi untuk menemukan pola terbaik menggunakan Dynamic Programming
def find_best_pattern(historical_data, days_to_predict):
    n = len(historical_data)
    # DP table untuk menyimpan skor pola terbaik
    dp = [[0] * (days_to_predict + 1) for _ in range(n)]
    # Rekonstruksi pola
    pattern = []

    # Mengisi DP table
    for i in range(1, n):
        for j in range(1, days_to_predict + 1):
            if i + j < n:
                dp[i][j] = max(dp[i][j], dp[i-1][j-1] + abs(historical_data[i] - historical_data[i+j]))

    # Menemukan pola dengan skor terbaik
    max_score = 0
    max_index = 0
    for i in range(n):
        if dp[i][days_to_predict] > max_score:
            max_score = dp[i][days_to_predict]
            max_index = i

    # Mengumpulkan pola terbaik
    for j in range(days_to_predict):
        pattern.append(historical_data[max_index + j])

    return pattern

In [15]:
# Fungsi untuk memprediksi suhu berdasarkan pola terbaik
def predict_temperature(historical_data, days_to_predict):
    pattern = find_best_pattern(historical_data, days_to_predict)
    last_known_temp = historical_data[-1]
    predicted_temperatures = []

    # Menggunakan pola untuk prediksi
    for i in range(days_to_predict):
        predicted_temp = pattern[i % len(pattern)] + (last_known_temp - pattern[0])
        predicted_temperatures.append(predicted_temp)
        last_known_temp = predicted_temp

    return predicted_temperatures

In [16]:
from datetime import datetime, timedelta
#code untuk menentukan prediksi data cuaca di hari berikutnya
def get_predictions(days_to_predict):
    predicted_temperatures = predict_temperature(temperatures, days_to_predict)
    start_date = datetime.now()
    predictions = []
    for i, temp in enumerate(predicted_temperatures):
        prediction_date = start_date + timedelta(days=i+1)
        predictions.append((prediction_date.strftime('%Y-%m-%d'), f"{temp:.2f}°C"))
    return predictions

In [17]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.9/315.9 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.5 MB/s 

In [19]:
import gradio as gr


# Define Gradio interface
def_interface = gr.Interface(fn=get_predictions, inputs="number", outputs="text", title="Temperature Prediction Model", description="Predict the temperature for the next few days.")
def_interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://351ffcafeb88a6a660.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
